# ELB의 구조
- Internet-Facing : VPC 와부의 인터넷에서 직접 접근할 수 있는 ELB
    - Public IP를 가진 노드를 AZ 내부에 생성하고 DNS로 접근
    - 주로 외부의 요청에 대한 트래픽을 분배할 때 활용
- Internal : VPC 내부에서만 접근할 수 있는 ELB
    - Private IP만을 가진 노드를 AZ 내부에 생성하고 DNS로 접근
    - 주로 VPC 내부의 레이어 단위로 트래픽을 배분할 때 활용

## ELB의 노드
- ELB가 생성될 경우 각 AZ및 서브넷에 퍼블릭, 프라이빗 아이피를 가진 노드 생성
    - 실제 요청은 이 노드를 통해 대상으로 전달
- 이후 트래픽 요청사항 필요에 따라 (ALB 기준) 최대 100개까지 증설(혹은 제거)
    - Scale 기준
        - ALB : 트래픽, bandwidth, 동시 연결 숫자 등 + WAF 혹은 Lambda 등의 처리를 위한 연산
            - 일반적으로 5분에 두배 증가 가능 (예: bandwidth 기준 5gbps 사용 시 5분 후 10gbps까지 증설)
        - NLB : bandwidth
            - 분당 3gbps씩 증가 가능
- ALB의 경우 각 서브넷에 최소 하나의 노드 생성 (기본적으로 두개의 서브넷이 있으니 노드도 기본으로 두개 생성)
- NLB의 경우 AZ별로 하나씩 생성
- DNS 요청이 들어오면 만들어진 노드의 IP 목록을 전달
    - 증감된 IP를 반영할 수 있도록 DNS의 TTL은 1분 권장
- AWS 권장 사항은 각 서브넷에 최소 8개 이상의 IP 확보
    - 즉 CIDR /27 이상 권장

## Cross-Zone Load Balancing
- 활성화 시 각 노드가 속한 AZ 외 다른 AZ의 대상으로 트래픽 분배 가능
- ALB는 기본 활성화. NLB, GWLB는 기본 Off

## ALB의 요청 분배
- 3가지 알고리즘
    - Round Robin : 순차적으로 모든 타겟에게 돌아가면서 분배
    - Least Outstanding Request : 현재 가장 적은 요청을 받고 있는 쪽으로 분배
        - 엄청난 트래픽이 몰릴 경우 제대로 동작하지 않을 수 있음
        - Outstanding Request : 응답이 완료되지 않은 요청
    - Weighted Random : 무작위 타겟에 요청 분배
        - Anomaly Mitigation(타겟의 오류 응답에 따라 확률을 조정) 활용 가능

## NLB의 요청 분배
- 요청의 정보를 기반으로 해쉬값을 만들어 타겟 그룹에 분배
    - TCP 6 Tuple : src ip, src port, dest ip, dest port, sequence #, protocol, SYN flag
    - UDP 5 Tuple : src ip, src port, dest ip, dest port, protocol